In [1]:
import numpy as np
import numba
import matplotlib.pyplot as plt

# Decision trees and random forests

In this notebook, I will build a decision tree algorithm from scratch. The idea of decision trees is to recursively find optimal splits in the data set along a hyperplane in the feature space. The optimal split is found by employing measures like the entropy.

In [14]:
from sklearn.datasets import load_iris
iris = load_iris()
data, labels = iris['data'], iris['target']

In [22]:
iris['target_names']

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [349]:
def shannon_entropy(prob):
    new_prob = prob[prob != 0.0]
    return - np.sum(new_prob * np.log(new_prob))

def split_data(data, feature_indx, split_param):
    left_indcs = np.argwhere(data[:, feature_indx] <= split_param).T[0]
    right_indcs = np.argwhere(data[:, feature_indx] > split_param).T[0]
    return left_indcs, right_indcs

def probabilities(left_indcs, right_indcs, labels):
    left_prob = np.array([np.count_nonzero(labels[left_indcs] == c) for c in range(3)]) / len(left_indcs)
    right_prob = np.array([np.count_nonzero(labels[right_indcs] == c) for c in range(3)]) / len(right_indcs)
    return left_prob, right_prob

def loss(data, labels, feature_indx, split_param):
    left_indcs, right_indcs = split_data(data, feature_indx, split_param)
    if len(left_indcs) == 0 or len(right_indcs) == 0:
        return np.float64('Inf')
    else:
        left_prob, right_prob = probabilities(left_indcs, right_indcs, labels)
        return (len(left_indcs) * shannon_entropy(left_prob) + len(right_indcs) *  shannon_entropy(right_prob)) / len(data)

def optimal_split(data, labels):
    losses = np.array([[loss(data, labels, ind, split) for split in data[:,ind]] for ind in range(4)])
    optima = np.nonzero(losses == np.min(losses))
    rnd_optimum_ind = np.random.choice(len(optima[0]))
    opt_feature_indx = optima[0][rnd_optimum_ind]
    opt_splt = data[optima[1][rnd_optimum_ind], opt_feature_indx]
    return opt_feature_indx, opt_splt

class Node:
    def __init__(self, feature_indx=None, splt=None, left=None, right=None, *, value=None):
        self.feature_indx = feature_indx
        self.splt = splt
        self.left = left
        self.right = right
        self.value = value

    def is_leaf(self):
        return self.value is not None or (self.left is None and self.right is None)

def build_tree(data, labels):
    if len(np.unique(labels)) == 1:
        return Node(value=labels[0])
    if len(labels) == 1:
        return Node(value=labels[0])

    opt_feature_indx, opt_splt = optimal_split(data, labels)

    left_mask = data[:,opt_feature_indx] <= opt_splt
    right_mask = ~left_mask

    left_child = build_tree(data[left_mask,:], labels[left_mask])
    right_child = build_tree(data[right_mask,:], labels[right_mask])

    return Node(feature_indx=opt_feature_indx, splt=opt_splt, left=left_child, right=right_child)

def predict_one(x, node):
    # Base case: if leaf, return stored value
    if node.is_leaf():
        return node.value
    
    # Otherwise follow the split
    if x[node.feature_indx] <= node.splt:
        return predict_one(x, node.left)
    else:
        return predict_one(x, node.right)
    
def tree_pred(tree, data):
    return np.array([predict_one(x, tree) for x in data])

In [368]:
tree = build_tree(data_train, labels_train)
labels_pred_tree = tree_pred(tree, data_test)

This works! Apparently, decision trees can be highly sensitive under adding or removing a data point. That is where random forests come into play that allow to create an ensemble of decision trees to find a better balance of predictions. Also, by splitting the data into training and test sets, we can study whether the random forests reduce variance.

In [367]:
rand_indcs = np.random.choice(150, 150, replace=False)
data_train = data[rand_indcs[:120],:]
labels_train = labels[rand_indcs[:120]]
data_test = data[rand_indcs[120:],:]
labels_test = labels[rand_indcs[120:]]

Let's say we do the bagging $100$ times and each time, we randomly choose $90$ data points:

In [354]:
def build_random_forest(data, labels, n_bags, n_samples):
    forest = []
    for b in range(n_bags):
        rand_indcs = np.random.choice(len(data), n_samples, replace=True)
        bagged_data = data[rand_indcs,:]
        bagged_labels = labels[rand_indcs]
        forest.append(build_tree(bagged_data, bagged_labels))
    return forest

def forest_pred(forest, data):
    pred_ensemble = np.zeros((len(forest), len(data)))
    for (t,tree) in enumerate(forest):
        pred_ensemble[t,:] = tree_pred(tree, data)
    pred = np.round(np.mean(pred_ensemble, axis=0))
    return pred

In [395]:
forest = build_random_forest(data_train, labels_train, 20, 110)
labels_pred_forest = forest_pred(forest, data_test)

Let's look at the accuracy scores of both algorithms:

In [396]:
print("Accuracy of the random forest prediction on the Iris data set is", 100 * (1 - np.count_nonzero(labels_pred_forest - labels_test) / len(labels_test)), "%")
print("Accuracy of the singe tree prediction on the Iris data set is", 100 * (1 - np.count_nonzero(labels_pred_tree - labels_test) / len(labels_test)), "%")

Accuracy of the random forest prediction on the Iris data set is 90.0 %
Accuracy of the singe tree prediction on the Iris data set is 86.66666666666667 %


It looks like the data set is too small or too clean to show the effect of the random forest and the reduced variance. 